In [1]:
import pickle
import numpy as np
import os, re
import torch
import pytorch_lightning as pl

In [2]:
with open("train_data.pkl", "rb") as f:
    train_data = pickle.load(f)
with open("test_data.pkl", "rb") as f:
    test_data = pickle.load(f)

In [3]:
class DictDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.embeddings = data["embeddings"]
        self.labels = data["labels"]
    
    def __len__(self):
        return len(self.embeddings)
    
    def __getitem__(self, idx):
        return np.array(self.embeddings[idx]), self.labels[idx]

def create_dataloader(d, batch_size=1):
    dataset = DictDataset(d)
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [4]:
train_dataset = create_dataloader(train_data, batch_size=64)
test_dataset = create_dataloader(test_data, batch_size=64)

In [5]:
class Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(768,2)
    def forward(self, x):
        output = self.l1(x)
        return output
    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0),-1)
        output = self(x)
        f = torch.nn.CrossEntropyLoss()
        loss = f(output, y)
        self.log("train_loss", loss)
        return loss
    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(), lr=1e-3)
        return opt

In [6]:
m = Model()

In [7]:
logger = pl.loggers.TensorBoardLogger("tb_logs", name="my_model")

In [8]:
trainer = pl.Trainer(max_epochs=10,
                    logger=logger)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
trainer.fit(m, train_dataset, test_dataset)

/home/ts777/datascience/BERT/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: tb_logs/my_model
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 1.5 K 
--------------------------------
1.5 K     Trainable params
0         Non-trainable params
1.5 K     Total params
0.006     Total estimated model params size (MB)
/home/ts777/datascience/BERT/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/da

Training: |                                                                    | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
